## 特徴量のカンペはこちら ⇒ https://docs.google.com/spreadsheets/d/1T59ieP110cFg8MfOtIbW2M60iWuSwnIf/edit?usp=share_link&ouid=111727008586725963366&rtpof=true&sd=true

## Kaggle用データのインポート

In [1]:
from google.colab import drive
drive.mount('/content/drive')

COMPETITION = "house-prices-advanced-regression-techniques"
WORK_DIR = "/content/drive/MyDrive/Colab Notebooks/kaggle"

import os

if not(os.path.exists(WORK_DIR)):
  os.mkdir(WORK_DIR)
os.chdir(WORK_DIR)

from google.colab import files
files.upload()

Mounted at /content/drive


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"droeloe","key":"c330037c8e955090c87e6872ec333b32"}'}

In [2]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c {COMPETITION} # copy from competition page!
!unzip -o {COMPETITION}
!rm {COMPETITION}.zip

  0% 0.00/199k [00:00<?, ?B/s]
100% 199k/199k [00:00<00:00, 34.3MB/s]
Archive:  house-prices-advanced-regression-techniques.zip
  inflating: data_description.txt    
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


## ライブラリのインポート

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
from numpy import mean, std
!pip install optuna
import optuna

import statsmodels.formula.api as smf
from statsmodels.stats import power
import statsmodels.api as sm
from scipy import stats
import math
import random
import warnings
warnings.filterwarnings('ignore')

from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder
from sklearn.preprocessing import StandardScaler, PowerTransformer
from sklearn.model_selection import StratifiedKFold, train_test_split, KFold
from sklearn.metrics import mean_squared_error

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import math

PALETTE=['lightcoral', 'lightskyblue', 'gold', 'sandybrown', 'navajowhite',
        'khaki', 'lightslategrey', 'turquoise', 'rosybrown', 'thistle', 'pink']
sns.set_palette(PALETTE) # seabornのデフォルトの色を変更する
BACKCOLOR = '#f6f5f5'

from scipy.special import boxcox1p
from IPython.core.display import HTML
from scipy.stats import norm, skew #for some statistics
import missingno as msno
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import mutual_info_regression
from sklearn.feature_selection import SelectKBest, SelectPercentile
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.model_selection import RepeatedKFold
from scipy.stats import skew, boxcox_normmax
from scipy.special import boxcox1p

from sklearn.neighbors import LocalOutlierFactor
from sklearn.base import BaseEstimator, TransformerMixin
from copy import deepcopy
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler

from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from mlxtend.regressor import StackingCVRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import StackingRegressor

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.6/390.6 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 7.8 MB/s eta 0:00:00


### User modules

In [4]:
# 並び替え検定用の関数
def perm_fun(x, nA, nB):
    n = nA + nB
    idx_B = set(random.sample(range(n), nB))
    idx_A = set(range(n)) - idx_B
    return x.loc[list(idx_B)].mean() - x.loc[list(idx_A)].mean()

In [5]:
# permutation test
def permutation_test(data, var, target):

  vA = data[var].dropna().unique()[0]
  vB = data[var].dropna().unique()[1]
  mean_a = data[data[var] == vA][target].mean()
  mean_b = data[data[var] == vB][target].mean()
  print(f"{target} mean value with {var}={vA}: {mean_a}")
  print(f"{target} mean value with {var}={vB}: {mean_b}")
  print(f"{target} observed difference between ({var}={vA}) and ({var}={vB}): {mean_b - mean_a}")
  nA = data[data[var] == vA].shape[0]
  nB = data[data[var] == vB].shape[0]

  random.seed(1)
  perm_diffs = pd.Series([perm_fun(data[target], nA, nB) for _ in range(1000)])
  perm_diffs = perm_diffs.dropna()

  fig, ax = plt.subplots(figsize=(5, 5))
  ax.hist(perm_diffs, bins=11, rwidth=0.9)
  ax.axvline(x = mean_b - mean_a, color='black', lw=2)
  ax.text(mean_b - mean_a, 10, 'Observed\ndifference', bbox={'facecolor':'white'})
  ax.set_xlabel(f'{var} differences')
  ax.set_ylabel('Frequency')

  plt.tight_layout()
  plt.show()

  perm_diffs = np.array(perm_diffs)
  print('p-value: ', (np.mean(perm_diffs > mean_b - mean_a)))

In [6]:
def multi_table(table_list):
    return HTML(
        f"<table><tr> {''.join(['<td>' + table._repr_html_() + '</td>' for table in table_list])} </tr></table>")

In [7]:
def cat_dist(data, var, hue, msg_show=True):
    total_cnt = data[var].count()
    f, ax = plt.subplots(1, 2, figsize=(25, 8))
    hues = [None, hue]
    titles = [f"{var}'s distribution", f"{var}'s distribution by {hue}"]

    for i in range(2):
        sns.countplot(data[var], edgecolor='black', hue=hues[i], linewidth=1, ax=ax[i], data=data)
        ax[i].set_xlabel(var, weight='bold', size=13)
        ax[i].set_ylabel('Count', weight='bold', size=13)
        ax[i].set_facecolor(BACKCOLOR)
        ax[i].spines['top'].set_visible(False)
        ax[i].spines['right'].set_visible(False)
        ax[i].set_title(titles[i], size=15, weight='bold')
        for patch in ax[i].patches:
            x, height, width = patch.get_x(), patch.get_height(), patch.get_width()
            if msg_show:
                ax[i].text(x + width / 2, height + 3, f'{height} \n({height / total_cnt * 100:2.2f}%)', va='center', ha='center', size=12, bbox={'facecolor': 'white', 'boxstyle': 'round'})
    plt.show()

In [8]:
def continuous_dist(data, x, y):
    f, ax = plt.subplots(1, 4, figsize=(35, 10))
    sns.histplot(data=train, x=y, hue=x, ax=ax[0], element='step')
    sns.violinplot(x=data[x], y=data[y], ax=ax[1], edgecolor='black', linewidth=1)
    sns.boxplot(x=data[x], y=data[y], ax=ax[2])
    sns.stripplot(x=data[x], y=data[y], ax=ax[3])
    for i in range(4):
        ax[i].spines['top'].set_visible(False)
        ax[i].spines['right'].set_visible(False)
        ax[i].set_xlabel(x, weight='bold', size=20)
        ax[i].set_ylabel(y, weight='bold', size=20)
        ax[i].set_facecolor(BACKCOLOR)
    f.suptitle(f"{y}'s distribution by {x}", weight='bold', size=25)
    plt.show()

In [9]:
# ANOVA using permutation test
# Image size が ～～～ のエラーが出た場合は logx = True に設定する
def anova_perm(data, var, target, logx=False):

  # Creating crosstab
  crosstb = pd.crosstab(data[var], data[target])

  # Creating barplot
  # pl = crosstb.plot(kind="bar", stacked=True, rot=0)

  for col in data[var].unique():
      try:
          true_ratio = len(data[(data[var]==col)&(data[target]==True)]) / len(data[data[var]==col])
      except:
          break
      print(col, true_ratio)

  observed_variance = data.groupby(var).mean().var()[target] # グループごとの平均値を求め、その平均値の分散を計算している
  print('Observed means:', data.groupby(var).mean()[target].values.ravel()) # ravel: 横方向の1次元のベクトルを返す
  print('Variance:', observed_variance)
  # Permutation test example with stickiness
  def perm_test(data):
      data = data.copy()
      data[target] = np.random.permutation(data[target].values)
      return data.groupby(var).mean().var()[target]

  random.seed(1)
  perm_variance = [perm_test(data) for _ in range(3000)]
  print('Pr(Prob)', np.mean([var > observed_variance for var in perm_variance]))

  if logx == True:
    # 横長のグラフになってしまうので自然対数を取った（通常は下記2行は不要）
    observed_variance_log = np.log(observed_variance)
    perm_variance_log = np.log(perm_variance)

    fig, ax = plt.subplots(figsize=(5, 5))
    ax.hist(perm_variance_log, bins=11, rwidth=0.9)

    # 横長のグラフになってしまうので自然対数を取った（通常はxにobserved_varianceを指定）
    ax.axvline(x = observed_variance_log, color='black', lw=2)
    ax.text(observed_variance, ax.get_ylim()[1]*0.8, 'Observed\nvariance', bbox={'facecolor':'white'})
    ax.set_xlabel('Variance')
    ax.set_ylabel('Frequency')

    plt.tight_layout()
    plt.show()
  else:
    fig, ax = plt.subplots(figsize=(5, 5))
    ax.hist(perm_variance, bins=11, rwidth=0.9)

    ax.axvline(x = observed_variance, color='black', lw=2)
    ax.text(observed_variance, ax.get_ylim()[1]*0.8, 'Observed\nvariance', bbox={'facecolor':'white'})
    ax.set_xlabel('Variance')
    ax.set_ylabel('Frequency')

    plt.tight_layout()
    plt.show()

In [10]:
# column_transformer 利用後のカラム名を取得する関数
def get_feature_names(column_transformer):
    """Get feature names from all transformers.
    Returns
    -------
    feature_names : list of strings
        Names of the features produced by transform.
    """
    # Remove the internal helper function
    #check_is_fitted(column_transformer)

    # Turn loopkup into function for better handling with pipeline later
    def get_names(trans):
        # >> Original get_feature_names() method
        if trans == 'drop' or (
                hasattr(column, '__len__') and not len(column)):
            return []
        if trans == 'passthrough':
            if hasattr(column_transformer, '_df_columns'):
                if ((not isinstance(column, slice))
                        and all(isinstance(col, str) for col in column)):
                    return column
                else:
                    return column_transformer._df_columns[column]
            else:
                indices = np.arange(column_transformer._n_features)
                return ['x%d' % i for i in indices[column]]
        if not hasattr(trans, 'get_feature_names'):
        # >>> Change: Return input column names if no method avaiable
            # Turn error into a warning
            warnings.warn("Transformer %s (type %s) does not "
                                 "provide get_feature_names. "
                                 "Will return input column names if available"
                                 % (str(name), type(trans).__name__))
            # For transformers without a get_features_names method, use the input
            # names to the column transformer
            if column is None:
                return []
            else:
                return [name + "__" + f for f in column]

        return [name + "__" + f for f in trans.get_feature_names()]

    ### Start of processing
    feature_names = []

    # Allow transformers to be pipelines. Pipeline steps are named differently, so preprocessing is needed
    if type(column_transformer) == Pipeline:
        l_transformers = [(name, trans, None, None) for step, name, trans in column_transformer._iter()]
    else:
        # For column transformers, follow the original method
        l_transformers = list(column_transformer._iter(fitted=True))


    for name, trans, column, _ in l_transformers:
        if type(trans) == Pipeline:
            # Recursive call on pipeline
            _names = get_feature_names(trans)
            # if pipeline has no transformer that returns names
            if len(_names)==0:
                _names = [name + "__" + f for f in column]
            feature_names.extend(_names)
        else:
            feature_names.extend(get_names(trans))

    return feature_names
# get_feature_names(preprocessor)

In [11]:
def scatter_hist(data, xlabel, ylabel):
  x = data[xlabel]
  y = data[ylabel]

  fig = plt.figure(figsize=(8,8))
  gs = gridspec.GridSpec(3, 3)
  ax_main = plt.subplot(gs[1:3, :2])
  ax_xDist = plt.subplot(gs[0, :2],sharex=ax_main)
  ax_yDist = plt.subplot(gs[1:3, 2],sharey=ax_main)

  ax_main.scatter(x,y,marker='.')
  ax_main.set(xlabel=xlabel, ylabel=ylabel)

  ax_xDist.hist(x,bins=100,align='mid')
  ax_xDist.set(ylabel='count')
  ax_xCumDist = ax_xDist.twinx()
  ax_xCumDist.hist(x,bins=100,cumulative=True,histtype='step',density=True,color='r',align='mid')
  ax_xCumDist.tick_params('y', colors='r')
  ax_xCumDist.set_ylabel('cumulative',color='r')

  ax_yDist.hist(y,bins=100,orientation='horizontal',align='mid')
  ax_yDist.set(xlabel='count')
  ax_yCumDist = ax_yDist.twiny()
  ax_yCumDist.hist(y,bins=100,cumulative=True,histtype='step',density=True,color='r',align='mid',orientation='horizontal')
  ax_yCumDist.tick_params('x', colors='r')
  ax_yCumDist.set_xlabel('cumulative',color='r')

  plt.show()

## データのダウンロード

In [12]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
submission = pd.read_csv("sample_submission.csv")

all_data = pd.concat([train, test], axis=0)

## target と 自動分類が上手くいかないカラムの設定

In [13]:
# target と 自動分類が上手くいかないカラムの設定
target = "SalePrice"
all_data["MSSubClass"] = all_data["MSSubClass"].astype("object")

### categorical や numerical の定義

In [14]:
def get_num_cat_column_names(df, target):
  feature_cols = df.columns.drop(target)
  categorical_cols = []
  numerical_cols = []
  for i in feature_cols:
      if df[i].dtype=='object':
          categorical_cols.append(i)
      else:
          numerical_cols.append(i)

  return categorical_cols, numerical_cols

### BoxCox変換する特徴量を決める

In [15]:
def get_skewed_column_names(df, numerical_cols):
  skewed_feats = df[numerical_cols].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
  skewness = pd.DataFrame({'Skew' :skewed_feats})
  skewness = skewness[skewness.abs() > 0.75]
  skewness.dropna(inplace=True)
  skewed_features = skewness.index.copy()

  new_numerical_cols = numerical_cols.copy()
  for col in skewed_features:
    new_numerical_cols.remove(col)

  return skewed_features, new_numerical_cols

In [16]:
def submit_results(model, train_X, train_y):
  model.fit(train_X, train_y)
  scores = cross_validate(model, train_X, train_y, scoring='neg_root_mean_squared_error', cv=cv, n_jobs=-1, error_score='raise', return_estimator=False) # model の状態を確認したい場合はreturn_estimator をTrueにする
  score = mean(scores["test_score"])
  print(f"score = {score}")
  y_pred = model.predict(test_X)
  output = pd.DataFrame({'Id': test["Id"],
                        'SalePrice': y_pred})

  path = "/content/drive/MyDrive/Colab Notebooks/data/output.csv"
  output.to_csv(path, index=False)
  display(output)

### 特徴量エンジニアリングの処理は一箇所にまとめる

In [17]:
def feature_engineering(data):
  # add new feature
  data["YearSinceRemodel"] = data["YrSold"] - data["YearBuilt"] # ここで新しい特徴量を追加する
  data['BsmtFinType1_Unf'] = 1*(data['BsmtFinType1'] == 'Unf') # ここで新しい特徴量を追加する
  data['HasWoodDeck'] = (data['WoodDeckSF'] == 0) * 1 # ここで新しい特徴量を追加する
  data['Total_Home_Quality'] = data['OverallQual'] + data['OverallCond'] # ここで新しい特徴量を追加する
  data['TotalSF'] = data['TotalBsmtSF'] + data['1stFlrSF'] + data['2ndFlrSF'] # ここで新しい特徴量を追加する
  data['has2ndfloor'] = data['2ndFlrSF'].apply(lambda x: 1 if x > 0 else 0) # ここで新しい特徴量を追加する
  data['hasgarage'] = data['GarageArea'].apply(lambda x: 1 if x > 0 else 0) # ここで新しい特徴量を追加する
  data['hasbsmt'] = data['TotalBsmtSF'].apply(lambda x: 1 if x > 0 else 0) # ここで新しい特徴量を追加する
  data['hasfireplace'] = data['Fireplaces'].apply(lambda x: 1 if x > 0 else 0) # ここで新しい特徴量を追加する
  data['haspool'] = data['PoolArea'].apply(lambda x: 1 if x > 0 else 0)
  data['YrBltAndRemod'] = data['YearBuilt'] + data['YearRemodAdd']
  data['Total_sqr_footage'] = (data['BsmtFinSF1'] + data['BsmtFinSF2'] + data['1stFlrSF'] + data['2ndFlrSF'])
  data['Total_Bathrooms'] = (data['FullBath'] + (0.5 * data['HalfBath']) + data['BsmtFullBath'] + (0.5 * data['BsmtHalfBath']))
  data['Total_porch_sf'] = (data['OpenPorchSF'] + data['3SsnPorch'] + data['EnclosedPorch'] + data['ScreenPorch'] + data['WoodDeckSF'])

  # delete features
  data = data.drop(['Utilities', 'Street', 'PoolQC',], axis=1)
  return data

In [18]:
def split_data(data, target):
  train_len = data.shape[0]-data[target].isna().sum()
  train_X = data.iloc[:train_len][data.columns.drop(target)]
  train_y = pd.DataFrame(data.iloc[:train_len][target], columns=[target])
  test_X = data.iloc[train_len:][data.columns.drop(target)]

  return train_X, train_y, test_X

## 外れ値に対する処理の定義

In [19]:
class CleanOutlier(BaseEstimator, TransformerMixin):
    def __init__(self, contamination=0, strategy="mean"):
        self.contamination = contamination
        self.strategy = strategy

    def fit(self, X, y=None):
        if self.contamination==0: return self
        self.lof = LocalOutlierFactor(contamination=self.contamination, novelty=True)
        self.lof.fit(X)
        return self

    def transform(self, X_):
        X = deepcopy(X_)
        if self.contamination==0: return X
        idx_outlier = self.lof.predict(X)==-1
        X[idx_outlier, :] = np.nan # set NaN to outlier value

        simple_imputer = SimpleImputer(strategy=self.strategy)
        X = simple_imputer.fit_transform(X) # fill outlier value usin simple_imputer

        return X

In [20]:
class SigmaOutlier(BaseEstimator, TransformerMixin):
    def __init__(self, sigma=3):
        self.sigma = sigma

    def fit(self, X, y=None):
        return self

    def transform(self, X_):
        X = deepcopy(X_)
        X = pd.DataFrame(X)
        for col in X.columns:
          m = np.mean(X[col])
          sd = np.std(X[col])
          X[X[col]>(sd*self.sigma+m)] = m + sd * self.sigma
          X[X[col]<(sd*self.sigma*-1+m)] = m - sd * self.sigma

        return X.to_numpy()

## 基本の処理

In [ ]:
all_data_fe = all_data.copy()

# feature engineering
all_data_fe =feature_engineering(all_data_fe)

# get list of column types
categorical_cols, numerical_cols = get_num_cat_column_names(all_data_fe, target)
skewed_features, numerical_cols = get_skewed_column_names(all_data_fe, numerical_cols)

# define preprocessor
numeric_transformer = Pipeline(steps=[('num_imputer', SimpleImputer(strategy="median")), ("scaler", StandardScaler()), ("outlier", CleanOutlier(contamination=0.001, strategy="mean"))])
skewed_transformer = Pipeline(steps=[('num_imputer', SimpleImputer(strategy="median")), ("scaler", StandardScaler()), ("outlier", CleanOutlier(contamination=0.001, strategy="mean")), ("power", PowerTransformer(method='yeo-johnson'))])
categorical_transformer = Pipeline(steps=[("cat_imputer", SimpleImputer(strategy="constant", fill_value="unknown")), ("encoder", OrdinalEncoder())])
preprocessor = ColumnTransformer(transformers=[("num", numeric_transformer, list(set(numerical_cols)-set(skewed_features))), ("cat", categorical_transformer, categorical_cols), ("skew", skewed_transformer, skewed_features)])

# get preprocessed data
all_data_preprocessed = pd.DataFrame(preprocessor.fit_transform(all_data_fe), columns=get_feature_names(preprocessor))
all_data_preprocessed[target] = list(all_data_fe[target])

# split the preprocessed data to train_X, train_y, test_X
train_X, train_y, test_X = split_data(all_data_preprocessed, target)

# Evaluate model
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
model = lgb.LGBMRegressor(**{
    'n_estimators': 2871,
    'reg_alpha': 0.09256633727458624,
    'reg_lambda': 1.5750906034116963,
    'colsample_bytree': 0.6,
    'subsample': 0.4,
    'learning_rate': 0.014,
    'max_depth': 10,
    'num_leaves': 511,
    'min_child_samples': 4,
    'min_data_per_groups': 57})
scores = cross_validate(model, train_X, train_y, scoring='neg_root_mean_squared_error', cv=cv, n_jobs=-1, error_score='raise', return_estimator=False) # model の状態を確認したい場合はreturn_estimator をTrueにする
score = mean(scores["test_score"])

print(score)

-26394.34275199718


# モデルの定義

In [ ]:
ridge = make_pipeline(RobustScaler(), Ridge()) # Ridgeモデル
lasso = make_pipeline(RobustScaler(), Lasso()) # Lassoモデル
elasticnet = make_pipeline(RobustScaler(), ElasticNet()) # ElasticNetモデル
svr = make_pipeline(RobustScaler(), SVR()) # SVRモデル
rf = RandomForestRegressor() # ランダムフォレストモデル
gbr = GradientBoostingRegressor() # 勾配ブースティングモデル
xgb = XGBRegressor() # XGBRegressorモデル
lightgbm = lgb.LGBMRegressor() # LGBMRegressorモデル

models = {
    "ridge": ridge,
    "lasso": lasso,
    "elasticnet": elasticnet,
    "svr": svr,
    "rf": rf,
    "gbr": gbr,
    "xgb": xgb,
    "lightgbm": lightgbm
}

## モデルごとの精度を確認する

In [ ]:
all_data_fe = all_data.copy()

# feature engineering
all_data_fe =feature_engineering(all_data_fe)

# get list of column types
categorical_cols, numerical_cols = get_num_cat_column_names(all_data_fe, target)
skewed_features, numerical_cols = get_skewed_column_names(all_data_fe, numerical_cols)

# define preprocessor
numeric_transformer = Pipeline(steps=[('num_imputer', SimpleImputer(strategy="median")), ("scaler", StandardScaler()), ("outlier", CleanOutlier(contamination=0.001, strategy="mean"))])
skewed_transformer = Pipeline(steps=[('num_imputer', SimpleImputer(strategy="median")), ("scaler", StandardScaler()), ("outlier", CleanOutlier(contamination=0.001, strategy="mean")), ("power", PowerTransformer(method='yeo-johnson'))])
categorical_transformer = Pipeline(steps=[("cat_imputer", SimpleImputer(strategy="constant", fill_value="unknown")), ("encoder", OrdinalEncoder())])
preprocessor = ColumnTransformer(transformers=[("num", numeric_transformer, list(set(numerical_cols)-set(skewed_features))), ("cat", categorical_transformer, categorical_cols), ("skew", skewed_transformer, skewed_features)])

# get preprocessed data
all_data_preprocessed = pd.DataFrame(preprocessor.fit_transform(all_data_fe), columns=get_feature_names(preprocessor))
all_data_preprocessed[target] = list(all_data_fe[target])

# split the preprocessed data to train_X, train_y, test_X
train_X, train_y, test_X = split_data(all_data_preprocessed, target)

df_score = {}
for model_name, model in models.items():
  cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
  scores = cross_validate(model, train_X, train_y, scoring='neg_root_mean_squared_error', cv=cv, n_jobs=-1, error_score='raise', return_estimator=False) # model の状態を確認したい場合はreturn_estimator をTrueにする
  score = mean(scores["test_score"])
  # print(f"{model_name}: {score}")
  df_score[model_name] = score

df_score = pd.Series(df_score)
df_score

ridge        -36710.267699
lasso        -36957.204060
elasticnet   -35595.348557
svr          -81239.392015
rf           -27532.101162
gbr          -25858.948794
xgb          -28350.277889
lightgbm     -28983.585047
dtype: float64

## Stacking した場合のスコアを確認する

In [ ]:
# ベースモデルのインスタンス化
base_models = [(model_name, model) for model_name, model in models.items()]

# メタモデルのインスタンス化
meta_model = LinearRegression()

# StackingRegressorのインスタンス化
stacking_model = StackingRegressor(estimators=base_models, final_estimator=meta_model)

scores = cross_validate(stacking_model, train_X, train_y, scoring='neg_root_mean_squared_error', cv=cv, n_jobs=-1, error_score='raise', return_estimator=False) # model の状態を確認したい場合はreturn_estimator をTrueにする
score = mean(scores["test_score"])
print(score)

-26939.7916917333


# 個別ベースモデルのパラメータ最適化

## GBRのハイパーパラメータチューニング
Best parameters:  {'learning_rate': 0.047645839728456346, 'max_depth': 6, 'min_samples_leaf': 1, 'min_samples_split': 9, 'n_estimators': 670}
Best score:  -25028.484067264708

In [ ]:
# Objective関数の定義
def objective(trial):
    # ハイパーパラメータの範囲
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.11),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10)
    }

    # モデルのインスタンス化
    gbr = GradientBoostingRegressor(**params)

    # クロスバリデーションにより精度を確認する
    scores = cross_validate(gbr, train_X, train_y, scoring='neg_root_mean_squared_error', cv=cv, n_jobs=-1, error_score='raise', return_estimator=False) # model の状態を確認したい場合はreturn_estimator をTrueにする
    score = mean(scores["test_score"])

    return score

# Optunaの最適化の実行
study_name = "2300624_gbr"
db_name = f'sqlite:///{study_name}optuna.db'

study = optuna.create_study(study_name = study_name, direction='maximize',
                            sampler=optuna.samplers.TPESampler(seed=500), storage=db_name, load_if_exists = True)
study.enqueue_trial({'n_estimators': 100, 'learning_rate': 0.1, 'max_depth': 3, 'min_samples_split': 2, 'min_samples_leaf': 1})  # 各パラメータの初期値を設定
study.optimize(objective, n_trials=50)

# 最適なハイパーパラメータとスコアの表示
best_params = study.best_params
best_score = study.best_value
print("Best parameters: ", best_params)
print("Best score: ", best_score)

# 既存のstudyを削除したい場合はこちらを実施
# optuna.delete_study(study_name=study_name, storage=db_name)

[I 2023-06-24 10:43:43,996] A new study created in RDB with name: 2300624_gbr
[I 2023-06-24 10:43:51,918] Trial 0 finished with value: -26196.86778291983 and parameters: {'learning_rate': 0.1, 'max_depth': 3, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}. Best is trial 0 with value: -26196.86778291983.
[I 2023-06-24 10:45:05,678] Trial 1 finished with value: -26857.368716050383 and parameters: {'learning_rate': 0.01159502327122883, 'max_depth': 8, 'min_samples_leaf': 1, 'min_samples_split': 12, 'n_estimators': 725}. Best is trial 0 with value: -26196.86778291983.
[I 2023-06-24 10:45:35,120] Trial 2 finished with value: -26208.03735168142 and parameters: {'learning_rate': 0.03293463850849263, 'max_depth': 5, 'min_samples_leaf': 4, 'min_samples_split': 8, 'n_estimators': 453}. Best is trial 0 with value: -26196.86778291983.
[I 2023-06-24 10:46:32,839] Trial 3 finished with value: -26693.58766658216 and parameters: {'learning_rate': 0.029313114006301794, 'max_depth':

Best parameters:  {'learning_rate': 0.047645839728456346, 'max_depth': 6, 'min_samples_leaf': 1, 'min_samples_split': 9, 'n_estimators': 670}
Best score:  -25028.484067264708


In [ ]:
# 既存のstudyを削除したい場合はこちらを実施
# optuna.delete_study(study_name=study_name, storage=db_name)

## GBR最適パラメータでの提出 Score: 0.13418


In [ ]:
params = {'learning_rate': 0.047645839728456346, 'max_depth': 6, 'min_samples_leaf': 1, 'min_samples_split': 9, 'n_estimators': 670}
model = GradientBoostingRegressor(**params)
submit_results(model, train_X, train_y)

score = -25231.0788111278


,Id,SalePrice
0,1461,132322.421612
1,1462,166157.232674
2,1463,181462.298919
3,1464,196163.098097
4,1465,180864.799125
...,...,...
1454,2915,83162.900656
1455,2916,81324.675237
1456,2917,167714.652083
1457,2918,112756.666100


## ridgeのハイパーパラメータチューニング
Best parameters:  {'alpha': 233.1962258682788}
Best score:  -35232.70878814724

In [ ]:
# Objective関数の定義
def objective(trial):
    # ハイパーパラメータの範囲
    params = {
        'alpha': trial.suggest_loguniform('alpha', 0.01, 1000000.0),
    }

    # モデルのインスタンス化
    model = make_pipeline(RobustScaler(), Ridge(**params)) # Ridgeモデル

    # クロスバリデーションにより精度を確認する
    scores = cross_validate(model, train_X, train_y, scoring='neg_root_mean_squared_error', cv=cv, n_jobs=-1, error_score='raise', return_estimator=False) # model の状態を確認したい場合はreturn_estimator をTrueにする
    score = mean(scores["test_score"])

    return score

# Optunaの最適化の実行
study_name = "2300624_ridge"
db_name = f'sqlite:///{study_name}optuna.db'

optuna.logging.set_verbosity(optuna.logging.WARNING)
study = optuna.create_study(study_name = study_name, direction='maximize',
                            sampler=optuna.samplers.TPESampler(seed=500), storage=db_name, load_if_exists = True)
study.enqueue_trial({'alpha': 0.1})  # 各パラメータの初期値を設定
study.optimize(objective, n_trials=500, show_progress_bar=True)


# 最適なハイパーパラメータとスコアの表示
best_params = study.best_params
best_score = study.best_value
print("Best parameters: ", best_params)
print("Best score: ", best_score)

# 既存のstudyを削除したい場合はこちらを実施
# optuna.delete_study(study_name=study_name, storage=db_name)

  0%|          | 0/500 [00:00<?, ?it/s]

Best parameters:  {'alpha': 233.1962258682788}
Best score:  -35232.70878814724


## Lassoのチューニング
{'alpha': 1353.478742358339}

In [ ]:
# Objective関数の定義
def objective(trial):
    # ハイパーパラメータの範囲
    params = {
        'alpha': trial.suggest_loguniform('alpha', 0.01, 1000000.0),
    }

    # モデルのインスタンス化
    model = make_pipeline(RobustScaler(), Lasso(**params)) # Lassoモデル

    # クロスバリデーションにより精度を確認する
    scores = cross_validate(model, train_X, train_y, scoring='neg_root_mean_squared_error', cv=cv, n_jobs=-1, error_score='raise', return_estimator=False) # model の状態を確認したい場合はreturn_estimator をTrueにする
    score = mean(scores["test_score"])

    return score

# Optunaの最適化の実行
study_name = "2300624_lasso"
db_name = f'sqlite:///{study_name}optuna.db'

optuna.logging.set_verbosity(optuna.logging.WARNING)
study = optuna.create_study(study_name = study_name, direction='maximize',
                            sampler=optuna.samplers.TPESampler(seed=200), storage=db_name, load_if_exists = True)
study.enqueue_trial({'alpha': 0.1})  # 各パラメータの初期値を設定
study.optimize(objective, n_trials=200, show_progress_bar=True)


# 最適なハイパーパラメータとスコアの表示
best_params = study.best_params
best_score = study.best_value
print("Best parameters: ", best_params)
print("Best score: ", best_score)

# 既存のstudyを削除したい場合はこちらを実施
# optuna.delete_study(study_name=study_name, storage=db_name)

  0%|          | 0/200 [00:00<?, ?it/s]

Best parameters:  {'alpha': 1353.478742358339}
Best score:  -34844.5255727275


## ElasticNetのチューニング
{'alpha': 16.936515650457828, 'l1_ratio': 0.987172818555732}

In [ ]:
# Objective関数の定義
def objective(trial):
    # ハイパーパラメータの範囲
    params = {
        'alpha': trial.suggest_loguniform('alpha', 0.01, 1000000.0),
        "l1_ratio":trial.suggest_uniform('l1_ratio', 0.0, 1.0)
    }

    # モデルのインスタンス化
    model = make_pipeline(RobustScaler(), ElasticNet(**params)) # ElasticNetモデル

    # クロスバリデーションにより精度を確認する
    scores = cross_validate(model, train_X, train_y, scoring='neg_root_mean_squared_error', cv=cv, n_jobs=-1, error_score='raise', return_estimator=False) # model の状態を確認したい場合はreturn_estimator をTrueにする
    score = mean(scores["test_score"])

    return score

# Optunaの最適化の実行
study_name = "2300624_elasticnet"
db_name = f'sqlite:///{study_name}optuna.db'

optuna.logging.set_verbosity(optuna.logging.WARNING)
study = optuna.create_study(study_name = study_name, direction='maximize',
                            sampler=optuna.samplers.TPESampler(seed=200), storage=db_name, load_if_exists = True)
study.enqueue_trial({'alpha': 0.1, "l1_ratio":0.5})  # 各パラメータの初期値を設定
study.optimize(objective, n_trials=200, show_progress_bar=True)


# 最適なハイパーパラメータとスコアの表示
best_params = study.best_params
best_score = study.best_value
print("Best parameters: ", best_params)
print("Best score: ", best_score)

# 既存のstudyを削除したい場合はこちらを実施
# optuna.delete_study(study_name=study_name, storage=db_name)

  0%|          | 0/200 [00:00<?, ?it/s]

Best parameters:  {'alpha': 16.936515650457828, 'l1_ratio': 0.987172818555732}
Best score:  -35226.01193150012


## SVRのチューニング
Best parameters:  {'C': 381116.7662099551, 'coef0': 0.3574236812873617, 'degree': 2, 'epsilon': 0.004782359979680847, 'kernel': 'poly'}
Best score:  -33663.3941650191

In [ ]:
# Objective関数の定義
def objective(trial):
    # ハイパーパラメータの範囲
    params = {
        'C': trial.suggest_loguniform('C', 0.01, 1000000.0),
        'epsilon': trial.suggest_loguniform('epsilon', 0.001, 1.0),
        'kernel': trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly']),
        'degree': 3,
        'gamma': "scale",
        'coef0': 0.0
    }

    if params['kernel'] == 'poly':
        params['degree'] = trial.suggest_int('degree', 2, 5)
        params['coef0'] = trial.suggest_uniform('coef0', 0.0, 1.0)
    elif params['kernel'] == 'rbf':
        params['gamma'] = trial.suggest_loguniform('gamma', 0.001, 10.0)
    elif params['kernel'] == 'linear':
        params['degree'] = trial.suggest_int('degree', 2, 5)
        params['gamma'] = trial.suggest_loguniform('gamma', 0.001, 10.0)
        params['coef0'] = trial.suggest_uniform('coef0', 0.0, 1.0)

    # モデルのインスタンス化
    model = make_pipeline(RobustScaler(), SVR(**params)) # SVRモデル

    # クロスバリデーションにより精度を確認する
    scores = cross_validate(model, train_X, train_y, scoring='neg_root_mean_squared_error', cv=cv, n_jobs=-1, error_score='raise', return_estimator=False) # model の状態を確認したい場合はreturn_estimator をTrueにする
    score = mean(scores["test_score"])

    return score

# Optunaの最適化の実行
study_name = "2300624_svr"
db_name = f'sqlite:///{study_name}optuna.db'

optuna.logging.set_verbosity(optuna.logging.WARNING)
study = optuna.create_study(study_name = study_name, direction='maximize',
                            sampler=optuna.samplers.TPESampler(seed=200), storage=db_name, load_if_exists = True)
study.enqueue_trial({'C': 5.6761331916034665, 'epsilon': 0.03809314529183056, 'kernel': 'linear'})
study.optimize(objective, n_trials=30, show_progress_bar=True)


# 最適なハイパーパラメータとスコアの表示
best_params = study.best_params
best_score = study.best_value
print("Best parameters: ", best_params)
print("Best score: ", best_score)

# 既存のstudyを削除したい場合はこちらを実施
# optuna.delete_study(study_name=study_name, storage=db_name)

  0%|          | 0/30 [00:00<?, ?it/s]

Best parameters:  {'C': 381116.7662099551, 'coef0': 0.3574236812873617, 'degree': 2, 'epsilon': 0.004782359979680847, 'kernel': 'poly'}
Best score:  -33663.3941650191


## RFのチューニング
Best parameters:  {'max_depth': 13, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 4, 'n_estimators': 883}
Best score:  -27853.00216551386

In [ ]:
# Objective関数の定義
def objective(trial):
    # ハイパーパラメータの範囲
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 20),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
        'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2']),
        'random_state': 0
    }
    # モデルのインスタンス化
    model = RandomForestRegressor(**params) # ランダムフォレストモデル

    # クロスバリデーションにより精度を確認する
    scores = cross_validate(model, train_X, train_y, scoring='neg_root_mean_squared_error', cv=cv, n_jobs=-1, error_score='raise', return_estimator=False) # model の状態を確認したい場合はreturn_estimator をTrueにする
    score = mean(scores["test_score"])

    return score

# Optunaの最適化の実行
study_name = "2300624_rf"
db_name = f'sqlite:///{study_name}optuna.db'

optuna.logging.set_verbosity(optuna.logging.WARNING)
study = optuna.create_study(study_name = study_name, direction='maximize',
                            sampler=optuna.samplers.TPESampler(seed=200), storage=db_name, load_if_exists = True)
# 初期値の設定
study.enqueue_trial({
    'n_estimators': 100,
    'max_depth': 6,
    'min_samples_split': 2,
    'min_samples_leaf': 1,
    'max_features': 'sqrt',
    'random_state': 0
})
study.optimize(objective, n_trials=100, show_progress_bar=True)


# 最適なハイパーパラメータとスコアの表示
best_params = study.best_params
best_score = study.best_value
print("Best parameters: ", best_params)
print("Best score: ", best_score)

# 既存のstudyを削除したい場合はこちらを実施
# optuna.delete_study(study_name=study_name, storage=db_name)

  0%|          | 0/100 [00:00<?, ?it/s]

Best parameters:  {'max_depth': 13, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 4, 'n_estimators': 883}
Best score:  -27853.00216551386


## XGboostのチューニング
Best parameters:  {'colsample_bytree': 0.8286653954594418, 'learning_rate': 0.012687685082100298, 'max_depth': 5, 'n_estimators': 652, 'subsample': 0.8907037871506849}
Best score:  -25286.909089187968

In [ ]:
# Objective関数の定義
def objective(trial):
    # ハイパーパラメータの範囲
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.1),
        'subsample': trial.suggest_uniform('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.6, 1.0),
        'random_state': 0
    }

    # モデルのインスタンス化
    model = XGBRegressor(**params) # XGBRegressorモデル

    # クロスバリデーションにより精度を確認する
    scores = cross_validate(model, train_X, train_y, scoring='neg_root_mean_squared_error', cv=cv, n_jobs=-1, error_score='raise', return_estimator=False) # model の状態を確認したい場合はreturn_estimator をTrueにする
    score = mean(scores["test_score"])

    return score

# Optunaの最適化の実行
study_name = "2300624_xgb"
db_name = f'sqlite:///{study_name}optuna.db'

optuna.logging.set_verbosity(optuna.logging.WARNING)
study = optuna.create_study(study_name = study_name, direction='maximize',
                            sampler=optuna.samplers.TPESampler(seed=200), storage=db_name, load_if_exists = True)
# 初期値の設定
study.enqueue_trial({
    'n_estimators': 100,
    'max_depth': 3,
    'learning_rate': 0.1,
    'subsample': 1.0,
    'colsample_bytree': 1.0,
    'random_state': 0
})
study.optimize(objective, n_trials=100, show_progress_bar=True)


# 最適なハイパーパラメータとスコアの表示
best_params = study.best_params
best_score = study.best_value
print("Best parameters: ", best_params)
print("Best score: ", best_score)

# 既存のstudyを削除したい場合はこちらを実施
# optuna.delete_study(study_name=study_name, storage=db_name)

  0%|          | 0/100 [00:00<?, ?it/s]

Best parameters:  {'colsample_bytree': 0.8286653954594418, 'learning_rate': 0.012687685082100298, 'max_depth': 5, 'n_estimators': 652, 'subsample': 0.8907037871506849}
Best score:  -25286.909089187968


### 最適パラメータでXGboost提出

In [ ]:
params = {'colsample_bytree': 0.8286653954594418, 'learning_rate': 0.012687685082100298, 'max_depth': 5, 'n_estimators': 652, 'subsample': 0.8907037871506849}
model = XGBRegressor(**params) # XGBRegressorモデル
submit_results(model, train_X, train_y)

score = -25286.909089187968


,Id,SalePrice
0,1461,130493.312500
1,1462,163509.812500
2,1463,182842.468750
3,1464,188618.812500
4,1465,185835.390625
...,...,...
1454,2915,84574.117188
1455,2916,85533.187500
1456,2917,156692.125000
1457,2918,110032.570312


## LightGBMのチューニング
Best parameters:  {'colsample_bytree': 0.6573901565999203, 'learning_rate': 0.008655370795209588, 'max_depth': 6, 'n_estimators': 572, 'num_leaves': 84, 'subsample': 0.7017195728963672}
Best score:  -27963.712257897343

In [ ]:
# Objective関数の定義
def objective(trial):
    # ハイパーパラメータの範囲
    params = {
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'num_leaves': trial.suggest_int('num_leaves', 10, 100),
        'subsample': trial.suggest_uniform('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.6, 1.0),
        'random_state': 0
    }

    # モデルのインスタンス化
    model = lgb.LGBMRegressor(**params) # LGBMRegressorモデル

    # クロスバリデーションにより精度を確認する
    scores = cross_validate(model, train_X, train_y, scoring='neg_root_mean_squared_error', cv=cv, n_jobs=-1, error_score='raise', return_estimator=False) # model の状態を確認したい場合はreturn_estimator をTrueにする
    score = mean(scores["test_score"])

    return score

# Optunaの最適化の実行
study_name = "2300624_lgbm"
db_name = f'sqlite:///{study_name}optuna.db'

optuna.logging.set_verbosity(optuna.logging.WARNING)
study = optuna.create_study(study_name = study_name, direction='maximize',
                            sampler=optuna.samplers.TPESampler(seed=200), storage=db_name, load_if_exists = True)
# 初期値の設定
study.enqueue_trial({
    'learning_rate': 0.1,
    'n_estimators': 100,
    'max_depth': -1,
    'num_leaves': 31,
    'subsample': 1.0,
    'colsample_bytree': 1.0,
    'random_state': 0
})
study.optimize(objective, n_trials=100, show_progress_bar=True)


# 最適なハイパーパラメータとスコアの表示
best_params = study.best_params
best_score = study.best_value
print("Best parameters: ", best_params)
print("Best score: ", best_score)

# 既存のstudyを削除したい場合はこちらを実施
# optuna.delete_study(study_name=study_name, storage=db_name)

  0%|          | 0/100 [00:00<?, ?it/s]

Best parameters:  {'colsample_bytree': 0.6573901565999203, 'learning_rate': 0.008655370795209588, 'max_depth': 6, 'n_estimators': 572, 'num_leaves': 84, 'subsample': 0.7017195728963672}
Best score:  -27963.712257897343


# Stacking

## モデルの定義

In [21]:
ridge = make_pipeline(RobustScaler(), Ridge(**{'alpha': 233.1962258682788})) # Ridgeモデル
lasso = make_pipeline(RobustScaler(), Lasso(**{'alpha': 1353.478742358339})) # Lassoモデル
elasticnet = make_pipeline(RobustScaler(), ElasticNet(**{'alpha': 16.936515650457828, 'l1_ratio': 0.987172818555732})) # ElasticNetモデル
svr = make_pipeline(RobustScaler(), SVR(**{'C': 381116.7662099551, 'coef0': 0.3574236812873617, 'degree': 2, 'epsilon': 0.004782359979680847, 'kernel': 'poly'})) # SVRモデル
rf = RandomForestRegressor(**{'max_depth': 13, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 4, 'n_estimators': 883}) # ランダムフォレストモデル
gbr = GradientBoostingRegressor(**{'learning_rate': 0.047645839728456346, 'max_depth': 6, 'min_samples_leaf': 1, 'min_samples_split': 9, 'n_estimators': 670}) # 勾配ブースティングモデル
xgb = XGBRegressor(**{'colsample_bytree': 0.8286653954594418, 'learning_rate': 0.012687685082100298, 'max_depth': 5, 'n_estimators': 652, 'subsample': 0.8907037871506849}) # XGBRegressorモデル
lightgbm = lgb.LGBMRegressor(**{'colsample_bytree': 0.6573901565999203, 'learning_rate': 0.008655370795209588, 'max_depth': 6, 'n_estimators': 572, 'num_leaves': 84, 'subsample': 0.7017195728963672}) # LGBMRegressorモデル

models = {
    "ridge": ridge,
    "lasso": lasso,
    "elasticnet": elasticnet,
    "svr": svr,
    "rf": rf,
    "gbr": gbr,
    "xgb": xgb,
    "lightgbm": lightgbm
}

## モデルごとの精度確認

In [24]:
all_data_fe = all_data.copy()

# feature engineering
all_data_fe =feature_engineering(all_data_fe)

# get list of column types
categorical_cols, numerical_cols = get_num_cat_column_names(all_data_fe, target)
skewed_features, numerical_cols = get_skewed_column_names(all_data_fe, numerical_cols)

# define preprocessor
numeric_transformer = Pipeline(steps=[('num_imputer', SimpleImputer(strategy="median")), ("scaler", StandardScaler()), ("outlier", CleanOutlier(contamination=0.001, strategy="mean"))])
skewed_transformer = Pipeline(steps=[('num_imputer', SimpleImputer(strategy="median")), ("scaler", StandardScaler()), ("outlier", CleanOutlier(contamination=0.001, strategy="mean")), ("power", PowerTransformer(method='yeo-johnson'))])
categorical_transformer = Pipeline(steps=[("cat_imputer", SimpleImputer(strategy="constant", fill_value="unknown")), ("encoder", OrdinalEncoder())])
preprocessor = ColumnTransformer(transformers=[("num", numeric_transformer, list(set(numerical_cols)-set(skewed_features))), ("cat", categorical_transformer, categorical_cols), ("skew", skewed_transformer, skewed_features)])

# get preprocessed data
all_data_preprocessed = pd.DataFrame(preprocessor.fit_transform(all_data_fe), columns=get_feature_names(preprocessor))
all_data_preprocessed[target] = list(all_data_fe[target])

# split the preprocessed data to train_X, train_y, test_X
train_X, train_y, test_X = split_data(all_data_preprocessed, target)

df_score = {}
for model_name, model in models.items():
  cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
  scores = cross_validate(model, train_X, train_y, scoring='neg_root_mean_squared_error', cv=cv, n_jobs=-1, error_score='raise', return_estimator=False) # model の状態を確認したい場合はreturn_estimator をTrueにする
  score = mean(scores["test_score"])
  # print(f"{model_name}: {score}")
  df_score[model_name] = score

df_score = pd.Series(df_score)
df_score

KeyboardInterrupt: ignored

## Stacking_model での精度確認

In [ ]:
# ベースモデルのインスタンス化
base_models = [(model_name, model) for model_name, model in models.items()]

# メタモデルのインスタンス化
meta_model = LinearRegression()

# StackingRegressorのインスタンス化
stacking_model = StackingRegressor(estimators=base_models, final_estimator=meta_model)

scores = cross_validate(stacking_model, train_X, train_y, scoring='neg_root_mean_squared_error', cv=cv, n_jobs=-1, error_score='raise', return_estimator=False) # model の状態を確認したい場合はreturn_estimator をTrueにする
score = mean(scores["test_score"])
print(score)

-26690.219021670462


## 推論結果の提出

In [ ]:
submit_results(stacking_model, train_X, train_y)

score = -26609.302611797302


,Id,SalePrice
0,1461,128541.194010
1,1462,162051.302391
2,1463,185298.563950
3,1464,193862.328696
4,1465,184860.185249
...,...,...
1454,2915,80283.684895
1455,2916,80071.747525
1456,2917,157362.352941
1457,2918,106086.364223
